In [16]:
!pip install jupyter-dash
!pip install dash-canvas==0.1.0

In [31]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dash_canvas import DashCanvas
import plotly.express as px
from jupyter_dash import JupyterDash

In [40]:
# Load Data
df = px.data.tips()
filename = 'https://drive.google.com/file/d/1hYD2JiDmR6qLFwUbb7pO2d8KEg0ToMU8/view?usp=sharing'
# Build App
app = JupyterDash(__name__)
app.layout = html.Div(html.Center([
    html.H1("My Math Solver",style={"color":"cornflowerblue", "textAlign":"center"}),
    html.Div(
      DashCanvas(id='canvas', 
                 filename=filename, 
                 hide_buttons=['zoom', 'line', 'select', 'rectangle'],
                 lineColor='black',
                 lineWidth=5,
                 width=400),
                 style={"margin":"auto","border":"2px solid black", "width": "400px"}
    ),
    html.Div(
        dcc.Input(
            id='text_field', 
            type='text',
            placeholder="Enter equation here!")
        ),
    html.Button('Submit', id='submit_button', n_clicks=0),
    html.Div(id='container-button-basic',
             children='Enter a equation and press submit'),
    html.Img(id='my-image', width=300),
  ]))

@app.callback(Output('my-image', 'src'),[Input('canvas', 'json_data')])
def update_data(string):
    if string:
        print("here")
        mask = parse_jsonstring(string, io.imread(filename, as_gray=True).shape)
    else:
        raise PreventUpdate
    return array_to_data_url((255 * mask).astype(np.uint8))

def update_output(n_clicks, value):
    return 'The input value was "{}" and the button has been clicked {} times'.format(
        value,
        n_clicks
    )

# Run app and display result inline in the notebook
app.run_server(mode='external')

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
# Do not RUN! production Server only!!!
# ! pip install pyngrok
# ! ngrok authtoken xxxxxxx

# from pyngrok import ngrok

# # Open Http tunnel to port
# public_url = ngrok.connect(port = '8050')
# public_url
# ngrok.kill()
